In [ ]:
#기존에 사용하던 프롬프트
old_context = """
Read the following dialog, and follow the instructions to summarize the conversation.
[Dialog]
{dialog}
"""
old_functions = [
    {
        "name": "summarize_conversation",
        "description": "Useful when you decide and craft a summary for the given conversation.",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": """
[Task]
Summarize the given conversation in Korean, making sure to capture the main points and tone of the discussion.
[How To]
1. Analyze the conversation text.
2. Capture the key points discussed and the user's intent.
3. Summarize the conversation in Korean.
[Response Rules]
1. The summary must be in Korean.
2. Keep the summary concise and focused on the main topics of the conversation.
"""
                }
            },
            "required": ["summary"]
        }
    }
]

In [3]:
context = """
Summarize the following dialog focusing only on the key information shared by the user.
[Dialog]
{dialog}
"""

functions = [
    {
        "name": "summarize_conversation",
        "description": "Creates a focused summary of the user's shared information from the conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": """
[Task]
Create a concise summary in Korean focusing only on the key information shared by the user.

[Rules]
1. Focus only on the actual information and preferences shared by the user
2. Exclude meta-information about the conversation structure
3. Don't mention that this was a conversation or dialogue
4. Omit references to the assistant/AI/비서
5. Write in a straightforward, factual style

[Example Format]
기분이 좋다고 했으며, 오늘 딸기가 들어간 크림 모카번을 먹었다. 커피는 잠을 못 자는 문제로 마시지 않았다. 소금빵을 좋아하며 몇 주 전에 먹었고, 다음에는 학교 근처 브레덴코에 가볼 계획이다.
"""
                }
            },
            "required": ["summary"]
        }
    }
]

In [4]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()  # .env 파일 불러오기
OPENAI_KEY = os.getenv("OPENAI_KEY")


# OpenAI 클라이언트 초기화
client = OpenAI(api_key=OPENAI_KEY)

# date_list.txt 파일 읽기
date_list_file = "date_list.txt"

# summary 폴더 생성 (없으면 생성)
summary_folder = "summary"
os.makedirs(summary_folder, exist_ok=True)

try:
    with open(date_list_file, "r", encoding="utf-8") as date_file:
        dates = [line.strip() for line in date_file if line.strip()]  # 빈 줄 제거

    for date in dates:
        file_path = f"conversation/conversation_{date}.json"  # 입력 파일 경로
        output_file = f"{summary_folder}/summary_{date}.json"  # 출력 파일 경로

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                conversation = json.load(f)

            # 대화 내용 추출
            dialog = "\n".join([f"{entry['role']}: {entry['content']}" for entry in conversation])

            # 대화 요약 요청
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": context.format(dialog=dialog)}
                ],
                tools=[{
                    "type": "function",
                    "function": functions[0]
                }],
                tool_choice={"type": "function", "function": {"name": "summarize_conversation"}}
            )

            # 요약 출력
            function_call = response.choices[0].message.tool_calls[0].function
            function_args = json.loads(function_call.arguments)

            summary = function_args['summary']
            print(f"날짜 {date}의 대화 요약:")
            print(f"요약: {summary}")

            # 요약 내용을 JSON 파일로 저장
            with open(output_file, "w", encoding="utf-8") as output_f:
                json.dump({"summary": summary}, output_f, ensure_ascii=False, indent=4)

            print(f"날짜 {date}의 요약이 '{output_file}' 파일에 저장되었습니다.\n")

        except FileNotFoundError:
            print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        except json.JSONDecodeError:
            print(f"오류: '{file_path}' 파일의 JSON 형식이 올바르지 않습니다.")
        except Exception as e:
            print(f"날짜 {date} 처리 중 오류 발생: {str(e)}\n")

except FileNotFoundError:
    print("오류: 'date_list.txt' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {str(e)}")

날짜 250111의 대화 요약:
요약: 사용자는 오늘 마라탕을 먹었으며, 푸주를 토핑으로 선택했다. 마라탕을 먹은 후에는 딸기 탕후루를 먹었다. 또한 딸기가 들어간 빵을 사 먹었으며, 특히 크림이랑 딸기 모카번을 좋아한다. 내일도 마라탕을 먹을 예정이다.
날짜 250111의 요약이 'summary/summary_250111.json' 파일에 저장되었습니다.

날짜 250112의 대화 요약:
요약: 사용자는 기분이 좋다고 말하며, 딸기가 들어간 크림 모카번을 먹었다. 그는 커피를 마시면 잠을 잘 못 자기 때문에 커피와 함께 먹지 않았다. 그는 또한 소금빵을 좋아하며, 몇 주 전에 마지막으로 먹은 것 같다. 그의 다음 목표는 학교 아래 브레덴코에서 빵을 먹는 것이다. 케이크도 좋아하며, 최근에 생크림 케이크를 먹었다고 했다.
날짜 250112의 요약이 'summary/summary_250112.json' 파일에 저장되었습니다.

날짜 250113의 대화 요약:
요약: 오늘 가장 기대되는 것은 점심에 '선재 업고 튀어라'라는 드라마를 보면서 밥을 먹는 것이다. 그 중에서도 캐릭터 '솔이'를 특히 좋아한다. 부엌에서는 엄마가 싸준 도시락을 먹을 계획이며, 도시락에는 마라탕이 들어있다. 매운 음식을 좋아하며, 다음에 도전해보고 싶은 음식으로는 떡볶이와 짬뽕을 생각하고 있다. 이번 주말에는 마라탕을 먹을 계획이며, 그 외에 교회를 가는 것도 계획에 있다. 교회에서는 예배드리는 것을 가장 좋아한다. 예배 후에는 친구들과 카페를 가거나, 집에서 쉬곤 한다. 카페에서는 주로 바스트 치즈 케이크와 딸기 라떼를 마신다.
날짜 250113의 요약이 'summary/summary_250113.json' 파일에 저장되었습니다.

